In [13]:
import numpy as np
import os, sys
import json

project_root = os.path.abspath("")  # alternative
if project_root[-12:] == 'LyoSavin2023':
    base_dir = project_root
else:
    base_dir = os.path.dirname(project_root)
sys.path.append(os.path.join(base_dir, 'core'))
sys.path.append(os.path.join(base_dir, 'core/utils'))

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt
import zarr

from utils import remove_all_ticks_and_labels

# training a model during the oscillation

In [7]:
# -------------------------- set model parameters -------------------------- #
model_name = 'dendritic-online-learning'
model_number = 1
num_steps = 100
forward_schedule = 'sine'
num_hidden = [8, 8, 7, 7]  # 4 layers
num_ambient_dims = 2
num_epochs = 2e5
manifold_type = 'swiss_roll'
manifold_noise_amount = 0
# manifold_rotation_angle = 'np.pi/4'
dataset_size = int(2e3)
batch_size = 128
learning_rate = 3e-4

pretrained_model = {
    'use_pretrained_model_weights': False,
    'use_checkpoint_weights': False,
    'checkpoint_epoch': 1490000,
    'model_name': model_name,
    'model_num': 1
}

# -------------------------- save model description -------------------------- #
description = {
    'model_name': model_name,
    'model_number': model_number,
    'num_steps': num_steps,
    'forward_schedule': forward_schedule,
    'num_hidden': num_hidden,
    'num_ambient_dims': num_ambient_dims,
    'num_epochs': f'{num_epochs:.1e}',
    'manifold_type': manifold_type,
    'manifold_noise_amount': manifold_noise_amount,
    # 'manifold_rotation_angle': manifold_rotation_angle,
    'dataset_size': f'{dataset_size:.0e}',
    'batch_size': batch_size,
    'learning_rate': f'{learning_rate:.0e}',
    'use_pretrained_model': pretrained_model['use_pretrained_model_weights'],
}
if pretrained_model['use_pretrained_model_weights']:
    description['pretrained_model_name'] = pretrained_model['model_name']
    description['pretrained_model_num'] = pretrained_model['model_num']
    if pretrained_model['use_checkpoint_weights']:
        description['pretrained_checkpoint_epoch'] = pretrained_model['checkpoint_epoch']

json_savedir = os.path.join(base_dir, 'core/model_description')
model_name_and_number = f'{model_name}_{model_number}'
json_name = f'{model_name_and_number}.json'
with open(os.path.join(json_savedir, json_name), 'w') as file:
    json.dump(description, file)

In [15]:
global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}', flush=True)

from prior_utils import forward_process
from utils import noise_estimation_loss, save_model_weights
from models import VariableDendriticCircuit
from dataset_utils import load_unimodal_data_nd, generate_2d_swiss_roll


def save_checkpoint(epoch, model_state_dict, optimizer_state_dict, loss, model_name, model_number):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model_state_dict,
        'optimizer_state_dict': optimizer_state_dict,
        'loss': loss
    }
    save_path = os.path.join(base_dir, 'core/saved_weights', f'{model_name}_{model_number}', f'epoch={epoch}')
    from pathlib import Path
    Path(save_path).mkdir(parents=True, exist_ok=True)
    torch.save(checkpoint, os.path.join(save_path, 'checkpoint.pt'))

# ------------------------------ define dataset ------------------------------ #
dataset = generate_2d_swiss_roll(dataset_size, rescaled=True, return_as_tensor=True)[1]

# -------------------------------- load model -------------------------------- #
model = VariableDendriticCircuit(hidden_cfg=num_hidden, num_in=num_ambient_dims, num_out=num_ambient_dims, bias=True) 

# -------------------- TRAINING - reverse diffusion process ------------------ #
coefs = forward_process(num_steps, device, forward_schedule)
betas, alphas, alphas_prod, alphas_prod_p, alphas_bar_sqrt, one_minus_alphas_prod_log, one_minus_alphas_prod_sqrt = coefs
alphas_bar_sqrt = alphas_bar_sqrt.to(device)
one_minus_alphas_prod_sqrt = one_minus_alphas_prod_sqrt.to(device)

# training set
dataset = dataset.to(device)

# define model
if pretrained_model['use_pretrained_model_weights']:
    if pretrained_model['use_checkpoint_weights']==False:
        from utils import load_model_weights
        pretrained_model_name = pretrained_model['model_name']
        pretrained_model_num = pretrained_model['model_num']
        print(f'taking weights from pretrained model {pretrained_model_name}_{pretrained_model_num}!')
        model = load_model_weights(model, pretrained_model_name, pretrained_model_num, device)
    elif pretrained_model['use_checkpoint_weights']==True:
        from utils import load_model_weights_from_chkpt
        model, num_steps, ambient_dims = load_model_weights_from_chkpt(pretrained_model['model_name'], pretrained_model['model_num'], epoch_number=pretrained_model['checkpoint_epoch'], device=device)
        print('model weights loaded from checkpoint!', flush=True)
        
model.to(device)

# training parameteres
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
if pretrained_model['use_pretrained_model_weights'] and pretrained_model['use_checkpoint_weights']==True:
    from utils import load_optimizer_state_dict
    optimizer = load_optimizer_state_dict(optimizer, pretrained_model['model_name'], pretrained_model['model_num'], epoch_number=pretrained_model['checkpoint_epoch'], device=device)
    print('optimizer state dict loaded from checkpoint!', flush=True)

run_dir = os.path.join(base_dir, 'demos/runs', f'{model_name}_{model_number}')
tb = SummaryWriter(run_dir)
start_time = time.time()

# start training
model.train()
for t in tqdm(range(int(num_epochs)), total=int(num_epochs), desc='Training model', unit='epochs', miniters=int(num_epochs)/1000, maxinterval=float("inf")):
    permutation = torch.randperm(dataset.size()[0], device=device)

    for i in range(0, dataset.size()[0], batch_size):
        # retrieve current batch
        indices = permutation[i:i+batch_size]
        batch_x = dataset[indices]
        
        # compute the loss
        loss = noise_estimation_loss(model, batch_x, num_steps, alphas_bar_sqrt, one_minus_alphas_prod_sqrt, device, norm='l2', has_class_label=False)
        # zero the gradients
        optimizer.zero_grad()
        # backward pass: compute the gradient of the loss wrt the parameters
        loss.backward()
        # call the step function to update the parameters
        optimizer.step()
    
    if t % int(1e4) == 0:
        save_checkpoint(t, model.state_dict(), optimizer.state_dict(), loss.item(), model_name, model_number)
    
    # write to tensorboard
    tb.add_scalar('Loss', loss.item(), t)
tb.flush()

end_time = time.time()
duration = end_time - start_time
duration_mins = duration / 60
print(f'training took {duration:.0f} seconds, which is {duration_mins:.2f} minutes.')

save_model_weights(model, model_name, model_number)

Using device: cuda


Training model:   0%|          | 0/200000 [00:00<?, ?epochs/s]

KeyboardInterrupt: 